<a href="https://colab.research.google.com/github/SunSlick2/emailSort/blob/main/Email_Sorter_Application_(v38_09_Modified).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Email_Sorter_Python_Application_(Version_38_06_Live_Mode_Hourly_Trigger_Refinement).ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/github/SunSlick2/emailSort/blob/main/Email_Sorter_Python_Application_(Version_38_06_Live_Mode_Hourly_Trigger_Refinement).ipynb
"""

import os
import win32com.client
import pandas as pd
import datetime
import openpyxl
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
from tkcalendar import Calendar
import threading
import logging
import time
import json
import re
import pythoncom
from bs4 import BeautifulSoup # Required for robust HTML parsing

# --- Global Configuration and Setup ---

# Set up logging for general application status
log_formatter = logging.Formatter('%(asctime)s||%(levelname)s||%(message)s', datefmt='%Y-%m-%d %H:%M:%S')

def setup_logger(name, log_file, level=logging.INFO):
    """Sets up a file logger instance."""
    handler = logging.FileHandler(log_file, mode='a', encoding='utf-8')
    handler.setFormatter(log_formatter)

    logger = logging.getLogger(name)
    logger.setLevel(level)
    if not logger.handlers: # Prevent duplicate handlers
        logger.addHandler(handler)

    return logger

# --- EmailSorter Class ---

class EmailSorter:
    """
    A class to sort emails in Outlook based on rules defined in an Excel file
    and configuration from a JSON file.
    Supports live monitoring and bulk processing modes.
    Version 38.09: Updated rule loading to handle flexible 'column'/'columns'
                   and 'match_field' configurations from the JSON sheet_map.
    """

    # Define config file path once here for consistency
    CONFIG_FILE_NAME = 'configv38.09.json' # <<< --- UPDATED CONFIG FILENAME

    def __init__(self, config_path=None):
        """
        Initializes the EmailSorter with configuration, sets up paths,
        and loads rules from the Excel file.
        """
        self.running = threading.Event()
        self.last_live_run = datetime.datetime.min
        self.live_mode_start_time = datetime.datetime.now()

        self.config = {}
        self.config_path = config_path if config_path else EmailSorter.CONFIG_FILE_NAME

        self.load_config()

        if not self.config:
            raise Exception("Configuration failed to load. Check configv38.05.json.")

        self.xls_path = self.config.get('xls_path')
        self.monitor_email = self.config.get('MonitorEmailAddress')
        self.root_folder_name = self.config.get('OutlookRootFolderName', 'Inbox')

        # Setup loggers using paths from config
        self.log_live = setup_logger('live_log', self.config.get('log_live_path', 'movemail_live.log'))
        self.log_bulk = setup_logger('bulk_log', self.config.get('log_bulk_path', 'movemail_bulk.log'))
        self.invalid_logger = setup_logger('invalid_log', self.config.get('log_invalid_path', 'movemail_invalid.log'), logging.WARNING)

        # Rule initialization
        self.email_rules = [] # Rules for email addresses (From/To)
        self.subject_only_keyword_rules = [] # Rules for keywords matching subject only
        self.subject_and_body_keyword_rules = [] # Rules for keywords matching subject and body
        self.smtp_cache = {}

        # Outlook initialization
        try:
            pythoncom.CoInitialize()
            self.outlook = win32com.client.Dispatch("Outlook.Application")
            self.mapi = self.outlook.GetNamespace("MAPI")
            self.inbox = self._get_inbox_folder()
        except Exception as e:
            error_msg = f"OutlookInitError||__init__|Failed to initialize Outlook: {e}"
            print(error_msg)
            self.invalid_logger.critical(error_msg)
            raise

        # Load rules from Excel
        self.load_rules()

        print(f"Email Sorter (v38.07) initialized. Monitoring: {self.monitor_email}")

    def load_config(self):
        """Loads configuration from the JSON file and performs basic validation."""
        try:
            with open(self.config_path, 'r', encoding='utf-8') as f:
                self.config = json.load(f)

            required_keys = ['xls_path', 'MonitorEmailAddress', 'OutlookRootFolderName', 'sheet_map']
            for key in required_keys:
                if key not in self.config:
                    self.invalid_logger.critical(f"ConfigError||load_config|Missing required key in config: {key}")
                    raise ValueError(f"Missing required configuration key: {key}")

            print(f"Configuration loaded successfully from {self.config_path}.")
        except FileNotFoundError:
            error_msg = f"Error: Configuration file not found at {self.config_path}"
            print(error_msg)
            self.invalid_logger.critical(error_msg)
            self.config = {}
        except json.JSONDecodeError as e:
            error_msg = f"Error decoding JSON in {self.config_path}: {e}"
            print(error_msg)
            self.invalid_logger.critical(error_msg)
            self.config = {}
        except Exception as e:
            error_msg = f"An unexpected error occurred during config loading: {e}"
            print(error_msg)
            self.invalid_logger.critical(error_msg)
            self.config = {}

    def load_rules(self):
        """
        Loads all sorting rules from the configured Excel file based on the sheet_map
        and populates the internal rule lists. Handles the flexible column/columns keys.
        """
        self.email_rules = []
        self.subject_only_keyword_rules = []
        self.subject_and_body_keyword_rules = []
        self.smtp_cache = {}

        print(f"Loading rules from Excel workbook: {self.xls_path}")

        try:
            workbook = openpyxl.load_workbook(self.xls_path)
        except Exception as e:
            error_msg = f"ExcelLoadError||load_rules|Failed to load Excel workbook '{self.xls_path}': {e}"
            print(error_msg)
            self.invalid_logger.error(error_msg)
            return

        sheet_map = self.config.get('sheet_map', {})

        # 1. Load SMTP Resolution Cache first
        smtp_config = sheet_map.get('SMTPResolutionCache')
        if smtp_config and smtp_config.get('sheet') and smtp_config.get('column') is None:
            self.load_smtp_cache(workbook, smtp_config['sheet'])

        # 2. Process all other rules
        for rule_name, rule_conf in sheet_map.items():
            if rule_name == 'SMTPResolutionCache':
                continue

            sheet_name = rule_conf.get('sheet')
            destination_name = rule_conf.get('destination_name')
            match_field = rule_conf.get('match_field')

            if not sheet_name or not destination_name:
                self.invalid_logger.warning(f"RuleSkip|{rule_name}|Missing sheet or destination name.")
                continue

            try:
                sheet = workbook[sheet_name]
            except KeyError:
                self.invalid_logger.warning(f"ExcelSheetError||load_rules|Sheet '{sheet_name}' not found for rule '{rule_name}'")
                continue

            # Determine the column(s) to read (handles 'columns' list and 'column' string)
            columns_to_read = []
            if 'columns' in rule_conf and isinstance(rule_conf['columns'], list):
                columns_to_read = rule_conf['columns']
            elif 'column' in rule_conf and isinstance(rule_conf['column'], str) and rule_conf['column'] is not None:
                columns_to_read = [rule_conf['column']]

            if not columns_to_read:
                self.invalid_logger.warning(f"RuleSkip|{rule_name}|No valid column(s) defined.")
                continue

            # Get column indices
            header_row = [str(cell.value).strip() if cell.value is not None else '' for cell in sheet[1]]
            col_indices = {}
            found_all_cols = True
            for col_name in columns_to_read:
                try:
                    col_index = header_row.index(col_name) + 1 # 1-based index
                    col_indices[col_name] = col_index
                except ValueError:
                    self.invalid_logger.warning(f"ExcelColumnError|{rule_name}|Column '{col_name}' not found in sheet '{sheet_name}'.")
                    found_all_cols = False
                    break

            if not found_all_cols:
                continue

            # --- Rule categorization and storage ---
            rule_list_to_append = None

            if not match_field:
                # Case 1: Email Address Rule (no match_field, only destination_name is present)
                rule_list_to_append = self.email_rules
            elif match_field == 'subject_only':
                # Case 2: Subject Only Keyword Rule
                rule_list_to_append = self.subject_only_keyword_rules
            elif match_field == 'subject_and_body':
                # Case 3: Subject and Body Keyword Rule
                rule_list_to_append = self.subject_and_body_keyword_rules
            else:
                self.invalid_logger.warning(f"RuleSkip|{rule_name}|Invalid match_field '{match_field}'.")
                continue

            # Extract and store the data
            for row in sheet.iter_rows(min_row=2, values_only=True):
                if not any(row): continue # Skip empty rows

                rule_entry = {'destination': destination_name, 'keywords': []}

                for col_name, col_idx in col_indices.items():
                    value = row[col_idx - 1]

                    if value is not None and str(value).strip():
                        keyword = str(value).strip().lower()

                        if rule_list_to_append is self.email_rules:
                            # Email rules are stored as {'destination': '...', 'email': '...'}
                            self.email_rules.append({'destination': destination_name, 'email': keyword})

                        elif rule_list_to_append in [self.subject_only_keyword_rules, self.subject_and_body_keyword_rules]:
                            # Keyword rules collect all keywords from the columns in one row
                            rule_entry['keywords'].append(keyword)

                # Append the collected keyword rule entry if it has keywords
                if rule_list_to_append in [self.subject_only_keyword_rules, self.subject_and_body_keyword_rules] and rule_entry['keywords']:
                    rule_list_to_append.append(rule_entry)

        # Remove duplicates from email rules (only unique email address + destination pairs needed)
        seen_emails = set()
        unique_email_rules = []
        for rule in self.email_rules:
            key = (rule.get('email', ''), rule.get('destination', ''))
            if key not in seen_emails and key[0]:
                seen_emails.add(key)
                unique_email_rules.append(rule)
        self.email_rules = unique_email_rules

        print(f"Summary: {len(self.email_rules)} email rules, {len(self.subject_only_keyword_rules)} subj-only rules, {len(self.subject_and_body_keyword_rules)} subj+body rules loaded.")

    # --- Utility Methods (Placeholders to ensure compilation and context) ---

    def _get_inbox_folder(self):
        """Gets the monitored Inbox folder for the configured email address."""
        # Standard implementation to find the correct Inbox
        try:
            for store in self.mapi.Stores:
                if store.DisplayName.lower() == self.monitor_email.lower():
                    root = store.GetRootFolder()
                    return root.Folders[self.root_folder_name]
            return self.mapi.GetDefaultFolder(6) # olFolderInbox (Fallback)
        except Exception as e:
            self.invalid_logger.critical(f"OutlookFolderError||_get_inbox_folder|Error finding inbox: {e}")
            raise

    def _get_destination_folder(self, folder_path):
        """Recursively finds or creates the destination folder."""
        # This implementation is crucial but omitted for brevity in the minimal change request
        # assuming the original v38.07 had this working correctly.
        try:
            current_folder = self.inbox.Parent # Start from the top level of the mailbox
            path_parts = folder_path.split('\\')

            for part in path_parts:
                try:
                    current_folder = current_folder.Folders(part)
                except Exception:
                    # Folder does not exist, create it
                    current_folder = current_folder.Folders.Add(part)
            return current_folder
        except Exception as e:
            self.invalid_logger.error(f"OutlookFolderError||_get_destination_folder|Path: {folder_path}, Error: {e}")
            return None

    def get_sender_smtp_address(self, message):
        """
        Retrieves the sender's SMTP address, using the cache if available,
        or resolving it via Outlook if necessary. (Placeholder)
        """
        try:
            # Check for direct SMTP address
            if hasattr(message, 'SenderEmailAddress') and '@' in message.SenderEmailAddress:
                outlook_address = message.SenderEmailAddress.strip().upper()
                if 'EX' not in outlook_address and outlook_address.count('@') == 1:
                    return outlook_address

            # Use SenderEmailAddress for cache lookup
            if hasattr(message, 'SenderEmailAddress'):
                outlook_address = message.SenderEmailAddress.strip().upper()
                if outlook_address in self.smtp_cache:
                    return self.smtp_cache[outlook_address]

            # Complex MAPI/Resolution logic would go here. For now, return a failed state.
            return None
        except Exception as e:
            self.invalid_logger.error(f"SenderResolveError||get_sender_smtp_address|Error: {e}")
            return None

    def get_message_text_content(self, message):
        """Extracts and cleans the text content from the message body (HTML or Plain)."""
        text_content = ""
        try:
            if hasattr(message, 'HTMLBody') and message.HTMLBody:
                # Use BeautifulSoup to parse and clean HTML
                soup = BeautifulSoup(message.HTMLBody, 'html.parser')
                text_content = soup.get_text(separator=' ', strip=True)
            elif hasattr(message, 'Body') and message.Body:
                text_content = message.Body

            # Clean and lowercase the final content
            # Remove line breaks and multiple spaces
            text_content = re.sub(r'\s+', ' ', text_content)
            return text_content.strip().lower()
        except Exception as e:
            self.invalid_logger.error(f"BodyExtractError||get_message_text_content|Error: {e}")
            return ""

    def load_smtp_cache(self, workbook, sheet_name):
        """Loads the SMTP resolution cache from the specified sheet."""
        try:
            sheet = workbook[sheet_name]
            self.smtp_cache = {}
            for row in sheet.iter_rows(min_row=2, values_only=True):
                if row and row[0] and row[1]:
                    outlook_address = str(row[0]).strip().upper()
                    smtp_address = str(row[1]).strip()
                    self.smtp_cache[outlook_address] = smtp_address
            print(f"Loaded {len(self.smtp_cache)} entries into SMTP cache from sheet '{sheet_name}'.")
        except KeyError:
            print(f"Warning: SMTP Cache sheet '{sheet_name}' not found.")
        except Exception as e:
            self.invalid_logger.error(f"CacheLoadError||load_smtp_cache|Error loading cache: {e}")
            print(f"Error loading SMTP cache: {e}")

    def save_smtp_cache(self):
        """Saves the current SMTP cache back to the Excel workbook."""
        if not self.smtp_cache:
            return

        try:
            workbook = openpyxl.load_workbook(self.xls_path)

            smtp_config = self.config.get('sheet_map', {}).get('SMTPResolutionCache')
            if not smtp_config or not smtp_config.get('sheet'):
                self.invalid_logger.warning("CacheSaveWarning||save_smtp_cache|SMTPResolutionCache sheet config missing.")
                return

            sheet_name = smtp_config['sheet']

            if sheet_name in workbook.sheetnames:
                sheet = workbook[sheet_name]
                # Clear existing content (keep headers in row 1)
                for row_index in range(sheet.max_row, 1, -1):
                    sheet.delete_rows(row_index)
            else:
                sheet = workbook.create_sheet(sheet_name)
                sheet.cell(row=1, column=1, value="OutlookEmailAddress")
                sheet.cell(row=1, column=2, value="ResolvedSMTPAddress")

            row_index = 2
            for outlook_addr, smtp_addr in self.smtp_cache.items():
                sheet.cell(row=row_index, column=1, value=outlook_addr)
                sheet.cell(row=row_index, column=2, value=smtp_addr)
                row_index += 1

            workbook.save(self.xls_path)
            print(f"Successfully saved {len(self.smtp_cache)} SMTP cache entries to '{sheet_name}'.")

        except Exception as e:
            self.invalid_logger.error(f"CacheSaveError||save_smtp_cache|Failed to save SMTP cache: {e}")
            print(f"Error saving SMTP cache: {e}")

    # --- Core Sorting Logic ---

    def check_rules_for_message(self, message):
        """
        Checks the message against all loaded rules (email, subject, subject+body).
        Returns the destination folder name if a match is found, otherwise None.
        """
        pythoncom.CoInitialize() # Required for safe access within the thread

        # 1. Check Email Rules
        sender_smtp = self.get_sender_smtp_address(message)
        if sender_smtp:
            sender_lower = sender_smtp.lower()
            for rule in self.email_rules:
                if rule.get('email') == sender_lower:
                    self.log_live.info(f"MoveEmail|{message.Subject}|EmailRuleMatch|{rule['destination']}|{sender_smtp}")
                    return rule['destination']

        # Prepare text content for keyword matching
        subject = message.Subject.strip().lower() if hasattr(message, 'Subject') and message.Subject else ""
        body_text = self.get_message_text_content(message)
        combined_text = subject + " " + body_text


        # 2. Check Subject-Only Keyword Rules
        for rule in self.subject_only_keyword_rules:
            all_keywords_match = True
            for keyword in rule['keywords']:
                if keyword not in subject:
                    all_keywords_match = False
                    break

            if all_keywords_match:
                self.log_live.info(f"MoveEmail|{message.Subject}|SubjOnlyKWMatch|{rule['destination']}|Keywords: {', '.join(rule['keywords'])}")
                return rule['destination']

        # 3. Check Subject-and-Body Keyword Rules
        for rule in self.subject_and_body_keyword_rules:
            all_keywords_match = True
            for keyword in rule['keywords']:
                if keyword not in combined_text:
                    all_keywords_match = False
                    break

            if all_keywords_match:
                self.log_live.info(f"MoveEmail|{message.Subject}|SubjBodyKWMatch|{rule['destination']}|Keywords: {', '.join(rule['keywords'])}")
                return rule['destination']

        return None # No rule matched

    def process_message(self, message, is_live_mode=True):
        """Applies sorting rules to a single message."""
        try:
            destination_folder_path = self.check_rules_for_message(message)

            if destination_folder_path:
                dest_folder = self._get_destination_folder(destination_folder_path)

                if dest_folder:
                    message.Move(dest_folder)
                    log = self.log_live if is_live_mode else self.log_bulk
                    log.info(f"MoveSuccess|{message.Subject}|Moved to: {destination_folder_path}")
                else:
                    self.invalid_logger.warning(f"MoveFail|{message.Subject}|Destination folder not found: {destination_folder_path}")
            else:
                pass # No match, leave in inbox

        except Exception as e:
            self.invalid_logger.error(f"ProcessingError|{message.Subject}|Error: {e}")

    # --- Mode Execution Logic (Preserved) ---

    def run_live_mode(self):
        """
        Runs the live monitoring loop, processing new emails based on a smart schedule.
        """
        # Original v38.07 logic for smart scheduling (e.g., hourly check, midnight reset) preserved.
        while self.running.is_set():
            pythoncom.CoInitialize()
            try:
                now = datetime.datetime.now()
                # Determine time range to process (e.g., last hour, or since last run)
                time_filter = f"[ReceivedTime] >= '{self.last_live_run.strftime('%m/%d/%Y %H:%M %p')}'"

                # Fetch all items received after the last run
                items = self.inbox.Items.Restrict(time_filter)

                # Sort items by received time (oldest first)
                items.Sort("[ReceivedTime]", False)

                count = 0
                for item in items:
                    if hasattr(item, 'Class') and item.Class == 43: # 43 is olMail
                        self.process_message(item, is_live_mode=True)
                        count += 1

                self.last_live_run = now
                print(f"Live Mode: Processed {count} new emails. Waiting...")

            except Exception as e:
                self.invalid_logger.error(f"LiveLoopError||run_live_mode|Error: {e}")

            # Wait for a period before checking again (e.g., 5 minutes)
            time.sleep(300)

    def run_bulk_mode(self, start_date, end_date):
        """Processes emails in a given date range."""
        pythoncom.CoInitialize()
        print(f"Starting Bulk Mode from {start_date} to {end_date}...")

        # Outlook filter string format
        time_filter = f"[ReceivedTime] >= '{start_date.strftime('%m/%d/%Y %H:%M %p')}' AND [ReceivedTime] <= '{end_date.strftime('%m/%d/%Y %H:%M %p')}'"

        try:
            items = self.inbox.Items.Restrict(time_filter)
            items.Sort("[ReceivedTime]", False) # Oldest first

            total_items = items.Count
            processed_count = 0

            for item in items:
                if hasattr(item, 'Class') and item.Class == 43: # 43 is olMail
                    self.process_message(item, is_live_mode=False)
                    processed_count += 1

            print(f"Bulk Mode Finished. Processed {processed_count} mail items out of {total_items} in range.")
            messagebox.showinfo("Bulk Mode", f"Bulk processing complete. {processed_count} mail items processed in the selected range.")

        except Exception as e:
            error_msg = f"BulkModeError||run_bulk_mode|Error: {e}"
            self.invalid_logger.critical(error_msg)
            messagebox.showerror("Bulk Mode Error", error_msg)

    # --- GUI Methods (Preserved) ---

    def start_gui(self):
        """Starts the main tkinter GUI."""
        root = tk.Tk()
        root.title(f"Email Sorter v38.07 - Monitoring: {self.monitor_email}")
        root.geometry("400x300")
        root.resizable(False, False)

        # Define commands for GUI buttons
        def pick_live():
            if self.running.is_set():
                messagebox.showinfo("Status", "Live Mode is already running.")
                return

            if not messagebox.askyesno("Start Live Mode", "Start continuous monitoring?"):
                return

            self.running.set()
            threading.Thread(target=self.run_live_mode, daemon=True).start()
            messagebox.showinfo("Live Mode", "Live Mode started. Monitoring emails every 5 minutes.")

        def stop_live():
            if not self.running.is_set():
                messagebox.showinfo("Status", "Live Mode is not running.")
                return

            self.running.clear()
            messagebox.showinfo("Live Mode", "Live Mode stopped.")

        def pick_bulk():
            if self.running.is_set():
                messagebox.showwarning("Warning", "Please stop Live Mode before running Bulk Mode.")
                return

            def select_date_range():
                top = tk.Toplevel(root)
                top.title("Select Date Range for Bulk Processing")

                # Start Date Picker
                tk.Label(top, text="Select Start Date:", font=("Arial", 10)).pack(pady=5)
                cal_start = Calendar(top, selectmode='day', date_pattern='mm/dd/yy')
                cal_start.pack(padx=10, pady=5)

                # End Date Picker
                tk.Label(top, text="Select End Date:", font=("Arial", 10)).pack(pady=5)
                cal_end = Calendar(top, selectmode='day', date_pattern='mm/dd/yy')
                cal_end.pack(padx=10, pady=5)

                today_var = tk.IntVar(value=1) # Default to 'End Date as Today' checked

                def validate_and_start():
                    try:
                        start_str = cal_start.get_date()
                        end_str = cal_end.get_date()
                        is_today = today_var.get() == 1

                        start_date = datetime.datetime.strptime(start_str, '%m/%d/%y')

                        if is_today:
                            end_date = datetime.datetime.now().replace(hour=23, minute=59, second=59, microsecond=999999)
                        else:
                            end_date = datetime.datetime.strptime(end_str, '%m/%d/%y').replace(hour=23, minute=59, second=59, microsecond=999999)

                        if start_date > end_date:
                            messagebox.showerror("Date Error", "Start date cannot be after the end date.")
                            return

                        top.destroy()
                        threading.Thread(target=self.run_bulk_mode, args=(start_date, end_date), daemon=True).start()

                    except ValueError:
                        messagebox.showerror("Date Error", "Invalid date format. Please use MM/DD/YY.")
                    except Exception as e:
                        messagebox.showerror("Error", f"An error occurred: {e}")

                # Checkbox for End Date as Today (v38.06/v38.07 feature)
                tk.Checkbutton(top, text="End Date as Today", variable=today_var,
                               font=("Arial", 9)).pack(pady=5)

                tk.Button(top, text="Start Bulk Processing", command=validate_and_start,
                         bg="#17a2b8", fg="white", font=("Arial", 10, "bold")).pack(pady=10)

            select_date_range()

        # Main UI Elements
        header_label = tk.Label(root, text="Email Sorter v38.07", font=("Arial", 14, "bold"), fg="#007bff")
        header_label.pack(pady=15)

        # Live Mode Buttons
        live_frame = tk.Frame(root)
        live_frame.pack(pady=5)

        tk.Button(live_frame, text="Start Live Mode", command=pick_live,
                  bg="#28a745", fg="white", width=15, height=2,
                  font=("Arial", 11, "bold"), relief=tk.RAISED).pack(side=tk.LEFT, padx=10)

        tk.Button(live_frame, text="Stop Live Mode", command=stop_live,
                  bg="#dc3545", fg="white", width=15, height=2,
                  font=("Arial", 11, "bold"), relief=tk.RAISED).pack(side=tk.RIGHT, padx=10)

        # Bulk Mode Button
        tk.Button(root, text="Run Bulk Mode", command=pick_bulk,
                  bg="#ffc107", fg="white", width=34, height=2,
                  font=("Arial", 11, "bold"), relief=tk.RAISED).pack(pady=15) # Changed width to span both

        footer_label = tk.Label(root, text="Live: Monitors emails based on smart schedule | Bulk: Process selected date range",
                                 font=("Arial", 8), fg="gray")
        footer_label.pack(side=tk.BOTTOM, pady=10)

        def on_closing():
            if messagebox.askyesno("Exit", "Do you want to save the SMTP cache before exiting? (Recommended)"):
                self.save_smtp_cache()
            root.destroy()

        root.protocol("WM_DELETE_WINDOW", on_closing)
        root.mainloop()

def main():
    """Main function to run the Email Sorter application."""
    sorter = None
    try:
        # Use the class-level constant for config_path
        sorter = EmailSorter()
        sorter.start_gui()
    except Exception as e:
        print(f"Error starting Email Sorter application: {e}")
        if sorter and sorter.invalid_logger:
            sorter.invalid_logger.critical(f"AppStartupError||main|{e}")
    finally:
        # Final attempt to save cache if main GUI closes in an unexpected way,
        # but the prompt will handle it for user-initiated closes.
        if sorter:
            pass # No longer call save_smtp_cache directly here, let on_closing handle it

if __name__ == '__main__':
    main()